# Tennisspielen bei verschiedenem Wetter

Für Aufgaben, die typischerweise in Excel gelöst werden, bietet sich in Python die Bibliothek pandas an.

In [ ]:
import pandas as pd

In Jupyter Notebooks ist das Fragezeichen als Operator dafür da, um Hilfe anzuzeigen.
Dies funktioniert für Module, aber auch für Variablen etc.

In [ ]:
?pd

Nun sollte unten ein Fenster aufgegangen sein.
Lesen Sie den ersten Absatz und schließen Sie das Fenster wieder.

Neben dieser Hilfe lässt sich für so ziemlich jede Fragestellung über google eine Lösung in der Dokumentation oder auf stackoverflow finden.

## Einlesen der Daten

Datengetrieben Projekte benötigen Datenquellen.
Insbesondere in kleineren Unternehmen sind dies oft klassische Datenbanken, Excel-Tabellen oder CSV-Dateien.
Diese tabellenartig strukturierten Daten lassen sich gut mit pandas einlesen.

In [ ]:
df = pd.read_csv("tennis.tsv", sep=" \t", engine="python")
df

Wenn wir wissen möchten, welche Spalte von welchem Datentyp ist, verwenden wir die API von pandas.
Mit der Methode `info` bekommt man einen Überblick über die interne Datenstruktur.
Der Typ `object` steht für einen Text, den pandas nicht semantisch interpretieren kann.

In [ ]:
df.info()

Die Methode `describe` erstellt [deskriptive Statistiken](https://de.wikipedia.org/wiki/Deskriptive_Statistik) für alle Spalten, für die dies möglich ist.
Für Text lässt sich bspw. kein Mittelwert berechnen.

In [ ]:
df.describe(include="all")

**Für alle Skalenniveaus**
- `count` bezeichnet die Anzahl der vorhandenen Einträge (d. h. sie sind nicht `None`, `pd.NaN` o. ä.).

**Für nominalskalierte Attribute**:
- `unique` beschreibt die Anzahl der unterschiedlichen Ausprägungen.
- `top` beschreibt die am häufigsten angetroffene Ausprägung.
- `freq` beschreibt die Häufigkeit der Ausprägung von `top`.

**Für kardinalskalierte Attribute**

- `mean` beschreibt das arithmetische Mittel und `std` die Standardabweichung.
- `min` und `max` beschreiben das Minimum und das Maximum
- mit `25%`, `50%` und `75%` werden die jeweiligen Quartile beschreiben - das Quartil `50%` entspricht dem Median.

Nun betrachten wir die nominalskalierten Attribute genauer:

In [ ]:
other_columns = set(df.columns) - set(["Day"])
other_columns

Nun gehen wir über die übrigen Zeilen und schauen, welcher Text wie häufig vorkommt.

In [ ]:
for column in other_columns:
    print("Attribut: ", column)
    print(df[column].value_counts())
    print()

<span style="color:blue; font-weight:bold">Aufgabe 1<span/>

Welches Skalenniveau wird für das jeweilige Attribut verwernden?
Nominalskala, Ordinalskala, Intervallskala, Verhältnisskala oder Absolutskala?

Antwort: ...

## Vorhersage

Wie können wir nun vorhersagen, unter welchen Umständen Tennis gespielt wird und unter welchen nicht? Gibt es ein Muster?

In [ ]:
df["Play Tennis?"].value_counts()

Es wird häufiger Tennis gespielt als nicht gespielt.
Daraus können wir uns nun ein dummes Vorhersagewerkzeug basteln:
Es wird einfach getippt, dass immer Tennis gespielt wird!
Damit liegen wir in 9 von 9 + 5 = 14 Fällen richtig.

In [ ]:
# Füge Spalte mit der dummen Vorhersage hinzu:
df = df.assign(stupid_guess=["Yes" for _ in range(len(df))])

df

Noch ist die Tabelle übersichtlich, aber gerade bei großen Datenmengen können wir nicht immer den gesamten Datensatz betrachten.
Wir müssen die Informationen in Zahlen zusammenfassen.
Dies betrifft nun auch unsere Vorhersage.
Wie gut ist sie?
Welche Zahl, welche Metrik kann dies gut ausdrücken?

Eine Ja/Nein-Entscheidung wird auch als [binäre Klassifikation](https://de.wikipedia.org/wiki/Beurteilung_eines_bin%C3%A4ren_Klassifikators) bezeichnet und erlaubt eine Vielzahl von Auswertungen.
Über das Problem muss bekannt sein bzw. definiert werden, was schlimmer ist:
Sollte man lieber einmal zu viel "ja" getippt haben oder muss auf jeden Fall jedes getippte "ja" ein Treffer sein?
Für das Tennisspielen ist es vielleicht egal, aber wenn es um Gesundheit oder juristische Urteile geht, ist dies anders.

In diesem Beispiel ist die Treffergenauigkeit (Englisch: Accuracy) geeignet.
Diese drückt aus, in wie viel Prozent der Fälle die Klassifikation richtig lag, sowohl mit "ja" als auch mit "nein".
Ein falsches Tippen hat keine schlimmen Konsequenzen für das Leben des\*r Tennisspieler\*in.

Die Berechnung müssen wir nicht selbst implementieren, dafür gibt es bereits eine Bibliothek in Python.

In [ ]:
import sklearn.metrics

In [ ]:
sklearn.metrics.accuracy_score(df["Play Tennis?"], df["stupid_guess"])

In ca. 64 % der Fälle liegen wir also damit richtig.

Wie würde es aussehen, wenn wir immer eine Münze werfen?
Dafür gibt es ein weiteres Python-Modul, das uns helfen kann.

In [ ]:
import random


def random_guess():
    return ["Yes" if random.random() > .5 else "No"
            for _ in range(len(df))]


random_guess()

In [ ]:
for i in range(1, 11):
    this_guess = random_guess()
    accuracy = sklearn.metrics.accuracy_score(df["Play Tennis?"], this_guess)
    print(f"{i:02}) Zufälliges Ergebnis: {accuracy:.02}")  # Schleifenzähler mit führenden Nullen (auf zwei Stellen),
                                                           # Genauigkeit auf die zweite Stelle gerundet

Wir sehen, dass bei so wenigen Münzwürfen manchmal ähnlich gute Ergebnisse wie bei der Variante "immer ja" herauskommen.
Mit anderen Worten: Unser Datensatz ist noch deutlich zu klein!

### Vorhersage mit einem Entscheidungsbaum

In [ ]:
import sklearn.tree
dt = sklearn.tree.DecisionTreeClassifier()

Zum Trainieren teilen wir die Daten in Eingabewerte und einem Zielwert auf.
Der Zielwert soll auf Basis der Eingabewerte vorhergesagt werden.

In [ ]:
eingabe = df[["Day", "Outlook", "Temperature", "Humidity", "Wind"]]
eingabe

In [ ]:
ziel = df[["Play Tennis?"]]
ziel

Nun wird der Zusammenhang zwischen Eingabe und der Beobachtung gelernt

In [ ]:
dt.fit(eingabe, ziel)

Die letzte Zelle sollte zum Fehler `ValueError: could not convert string to float: 'Sunny'` geführt haben.
Hier müssen wir für die Methode `fit` die Daten vorverarbeiten.
Denn leider kann der Entscheidungsbaum nur mit Fließkommazahlen (floats) arbeiten.

Eine Möglichkeit ist das sogenannte One-Hot-Enkodieren.
Dafür werden alle vorhandenen Kategorien eines Attributs als eine eigene Spalte aufgenommen.
Indirekt modellieren wir somit jedes Attribut (außer den Tag) als nominalskalierte Variable.

Die Umrechnung von Kategorien in One-Hot-enkodierte Dateien wird bereits mit pandas ausgeliefert.
Es werden auch automatisch nur die Attribute enkodiert, die dies benötigen, siehe das Attribut `Day`!

In [ ]:
one_hot_data = pd.get_dummies(df[["Day", "Outlook", "Temperature", "Humidity", "Wind"]])

one_hot_data

<span style="color:blue; font-weight:bold">Aufgabe 2</span>

Rufen Sie nun die Methode `fit` mit den vorverarbeiteten Daten auf.

In [ ]:
# Schreiben Sie den Code in diese Zelle

Nun kann der Entscheidungsbaum mit der Methode `predict` für neue Beobachtungen vorhersagen, ob es ein Tag zum Tennisspielen ist oder nicht.
Weil wir keine weiteren Beobachtungen haben, überprüfen wird dies nun mit den bisherigen Daten.
Dafür lassen wir uns für jede Zeile in der Tabelle die Vorhersage ausgeben.

In [ ]:
y_pred = dt.predict(one_hot_data)
y_pred

In [ ]:
sklearn.metrics.accuracy_score(df["Play Tennis?"], y_pred)

Uuups, das ist aber ganz schön gut, ganze 100 % der Fälle lagen wir genau richtig.
Kann man den Nachbarn wirklich so leicht durchschauen?

<span style="color:blue; font-weight:bold">Aufgabe 3</span>

Stellen Sie eine Hypothese auf, warum der Entscheidungsbaum alles korrekt vorhergesagt hat.

Hypothese: ...

Wir können uns den Entscheidungsbaum auch visualisieren lassen.

In [ ]:
import matplotlib.pyplot as plt
from sklearn import tree
fig, ax = plt.subplots(figsize=(20, 10))
tree.plot_tree(dt, ax=ax, feature_names=one_hot_data.columns)
plt.show()

Hier ein Leitfaden zur Interpretation:
- In der obersten Zeile steht mit `<=` der Vergleich, nach dem nach links (zutreffend) und rechts (nicht zutreffend) aufgeteilt wird.
- Der [Gini-Koeffizient](https://de.wikipedia.org/wiki/Gini-Koeffizient) beträgt 0, wenn alle Einträge am Knoten gleich sind. Je größer der Wert ist, desto mehr unterschiedliche Einträge gibt es.
- `samples` steht für die Anzahl der Einträge
- value ist ein Tupel nach dem Muster \[Anzahl NO, Anzahl YES\].

<span style="color:blue; font-weight:bold">Aufgabe 4</span>

Frage:
Glauben Sie, dass dieser Entscheidungsbaum gut auf neue Beobachtungen in dem Folgetagen übertragen werden kann? Warum (nicht)?

Antwort: ...

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Lizenzvertrag" style="border-width:0; display:inline" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a> &nbsp;&nbsp;&nbsp;&nbsp;Dieses Werk von Marvin Kastner ist lizenziert unter einer <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Namensnennung 4.0 International Lizenz</a>.